In [1]:
import numpy as np

In [2]:
class MatrixFactorization(object):
    def __init__(self, data, K):
        '''
        Arguments:
        - data    : 2 dimensional rating matrix
        - K       : number of latent dimensions
        '''
        
        self.R = np.matrix(data)
        self.D = np.zeros( self.R.shape )
        self.K = K
        
        self.U = np.random.uniform( size=(self.R.shape[0], K) )
        self.P = np.random.uniform( size=(K, self.R.shape[1]) )
    
    def _compure_error(self):
        self.D = (self.R - self.estimate_all())
        
        return self.D
    
    def train(self, alpha=0.1, beta=0.02, iterations=1000):
        '''
        Arguments:
        - alpha   : learning-rate 
        - beta    : regularization-rate
        '''
        
        for _ in range(iterations):
            self._compure_error()
            
            U = self.U
            P = self.P
            
            for i in range(self.R.shape[0]):      
                for j in range(self.R.shape[1]):
                    for k in range(self.K):
                        ik = alpha * ( P[k, j] * self.D[i, j] + beta * U[i, k])
                        kj = alpha * ( U[i, k] * self.D[i, j] + beta * P[k, j])
                        if np.isfinite(ik):
                            self.U[i, k] += ik
                        if np.isfinite(kj):
                            self.P[k, j] += kj
            
            #non-negativity
            self.U = self.U.clip(min=0)
            self.P = self.P.clip(min=0)
            
        return np.nansum(np.nansum(abs(self._compure_error())))
    
    def estimate_all(self):
        return self.U.dot(self.P)
    
    def estimate(self, x, y):
        return self.U[x, :].dot(self.P[:, y])

In [3]:
#test data

R = np.array([
    [5, 3, np.NaN, 1],
    [4, np.NaN, np.NaN, 1],
    [1, 1, np.NaN, 5],
    [1, np.NaN, np.NaN, 4],
    [np.NaN, 1, 5, 4],
])

In [4]:
MF = MatrixFactorization(data=R, K=2)

In [14]:
error = MF.train(alpha=0.001, beta=0.0002, iterations=10000)
print('error:', error)

error: 0.00296703579419


In [15]:
print('estimated matrix:')
MF.estimate_all()

estimated matrix:


array([[ 5.00045812,  3.00033639,  3.87416542,  0.99996082],
       [ 4.00035669,  2.41693733,  3.31757522,  0.99999806],
       [ 0.99995421,  1.00015325,  6.01260596,  5.00059617],
       [ 1.0000441 ,  0.91682481,  4.92126818,  4.00031527],
       [ 1.14238082,  0.99984231,  5.00031073,  4.00015174]])

In [16]:
print('User feature matrix:')
MF.U

User feature matrix:


array([[ 0.13289104,  2.27632664],
       [ 0.19918949,  1.82105685],
       [ 2.25556431,  0.45520278],
       [ 1.79111615,  0.45524369],
       [ 1.78160694,  0.52003873]])

In [17]:
print('product feature matrix:')
MF.P

product feature matrix:


array([[ 0.        ,  0.17952905,  2.34988937,  2.15372523],
       [ 2.19672257,  1.30757973,  1.56475178,  0.31355344]])